# 1. 데이터 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')
wine.head()
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
print(train_data.shape, test_data.shape)

(5197, 3) (1300, 3)


# 2. Review

## 2-1. RandomForestClassifier

In [2]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_data, train_target)

from sklearn.model_selection import cross_validate
scores = cross_validate(rf, train_data, train_target,
                       return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score'])) # overfitting
print(rf.oob_score_)
print(rf.feature_importances_)

print(rf.score(test_data, test_target))

0.9973541965122431
0.8905151032797809
0.8934000384837406
[0.23167441 0.50039841 0.26792718]
0.8892307692307693


## 2-2. ExtraTreeClassifier

In [3]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
et.fit(train_data, train_target)

from sklearn.model_selection import cross_validate
scores = cross_validate(et, train_data, train_target
                       , return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # overfitting
print(et.feature_importances_)
print(et.score(test_data, test_target))

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]
0.8861538461538462


# 3. Gradient Boosting 모델 학습
- 깊이가 얕은 결정 트리를 사용해 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
    - default `max_depth = 3, n_estimators = 100`: 깊이가 3인 트리 100개
    - 과대적합에 강하고, 높은 일반화 성능!
    - 경사하강법을 사용해 트리를 앙상블에 추가한다.
    - 학습률 `learning_rate` 매개변수(default=1)로 속도를 조절한다.

## 3-1. 첫 번째 Gradient Boosting

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
gb.fit(train_data, train_target)

GradientBoostingClassifier(random_state=42)

**교차 검증 `cross_validate()` 함수 이용**

In [5]:
from sklearn.model_selection import cross_validate
scores = cross_validate(gb, train_data, train_target
                       , return_train_score = True, n_jobs = -1)

**검증 세트의 정확도 확인**

In [6]:
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # 거의 과대적합되지 않았다. 결정 트리 개수를 늘려도 강함.

0.8881086892152563 0.8720430147331015


## 3-2. 두 번째 Gradient Boosting
- 결정 트리 개수를 늘려서 다시 한 번 `n_estimators` 매개변수 사용

In [7]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2
                               , random_state = 42)
scores = cross_validate(gb, train_data, train_target
                       , return_train_score = True, n_jobs = -1)

**검증 세트의 정확도 확인**

In [8]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
    # 트리 개수를 늘려도 과대적합되지 않고 검증 세트 정확도 점수를 유지하고 있다.

0.9464595437171814 0.8780082549788999


**특징 중요도 `feature_importances`**

In [9]:
print(gb.feature_importances_)

NotFittedError: This GradientBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## 3-3. 테스트 세트의 정확도 확인

In [ ]:
gb.score(test_data, test_target)

# 4. 히스토그램 기반 그레이디언트 부스팅
- 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘
- 입력 특성을 256개 구간으로 나누어 노드를 분할할 때 최적의 분할을 빠르게 찾을 수 있다.
- `max_iter` 사용

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)

**교차 검증 `cross_validate()` 함수 이용**

In [ ]:
from sklearn.model_selection import cross_validate
score = cross_validate(hgb, train_data, train_target
                      , return_train_score = True)

**검증 세트의 정확도 확인**

In [ ]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

## 4-1. 특징 중요도 `permutation_importance`
- 반복해서 얻은 특성 중요도(importances), 평균(importances_mean), 표준 편차(importances_std)

**훈련 세트의 특성 중요도 계산**

In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_data, train_target)
result = permutation_importance(hgb, train_data, train_target
                               , n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

**테스트 세트의 특성 중요도 계산**

In [ ]:
result = permutation_importance(hgb, test_data, test_target
                               , n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

## 4-2. 테스트 세트의 정확도 확인

In [ ]:
hgb.score(test_data, test_target)

# 5. XGBoost
- 다양한 부스팅 알고리즘을 지원하는 라이브러리
- `tree_method = 'hist'` 지정하면 히스토그램 기반 그레이디언트 부스팅 사용할 수 있다.

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)

In [ ]:
scores = cross_validate(xgb, train_data, train_target
                       , return_train_score = True)

**검증 세트의 정확도 확인**

In [ ]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 6. LightGBM

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)

In [ ]:
scores = cross_validate(lgb, train_data, train_target
                       , return_train_score = True, n_jobs = -1)

**검증 세트의 정확도 확인**

In [ ]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))